In [9]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
plt.rcParams["figure.dpi"] = 300
np.set_printoptions(precision=3, suppress=True)
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Activation

In [43]:
from sklearn import datasets
iris = datasets.load_iris()
data = pd.DataFrame(data=np.c_[iris['data'],iris['target']],columns= iris['feature_names']+['target'])

In [44]:
X = data.iloc[:,0:4].astype(float)
Y = data.iloc[:,4].astype(int)

In [45]:
# Convert labels to categorical one-hot encoding
import keras
binary_Y = keras.utils.to_categorical(Y, num_classes=3)

In [83]:
model = Sequential()
model.add(Dense(4, kernel_initializer='normal', activation='relu', input_dim=4))
model.add(Dense(4, kernel_initializer='normal', activation='relu'))
model.add(Dense(3, kernel_initializer='normal', activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [84]:
X = np.asarray(X)
binary_Y = np.asarray(binary_Y)
X_train, X_test, y_train, y_test = train_test_split(X, binary_Y, stratify=binary_Y, random_state = 0)

In [85]:
model.fit(X_train, y_train,batch_size=5, epochs=100, verbose=1)

Epoch 1/100
112/112 [==============================] - 5s - loss: 1.0987 - acc: 0.3036     
Epoch 2/100
112/112 [==============================] - 0s - loss: 1.0978 - acc: 0.3839     
Epoch 3/100
112/112 [==============================] - 0s - loss: 1.0957 - acc: 0.5000     
Epoch 4/100
112/112 [==============================] - 0s - loss: 1.0912 - acc: 0.6875     
Epoch 5/100
112/112 [==============================] - 0s - loss: 1.0844 - acc: 0.4286     
Epoch 6/100
112/112 [==============================] - 0s - loss: 1.0707 - acc: 0.6875     
Epoch 7/100
112/112 [==============================] - 0s - loss: 1.0495 - acc: 0.6786     
Epoch 8/100
112/112 [==============================] - 0s - loss: 1.0193 - acc: 0.6875     
Epoch 9/100
112/112 [==============================] - 0s - loss: 0.9828 - acc: 0.6875     
Epoch 10/100
112/112 [==============================] - 0s - loss: 0.9433 - acc: 0.6875     
Epoch 11/100
112/112 [==============================] - 0s - loss: 0.8988 - acc

In [86]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss: {:.3f}".format(score[0]))
print("Test Accuracy: {:.3f}".format(score[1]))

Test loss: 0.579
Test Accuracy: 0.684


In [63]:
# model selection by grid search

from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV

def make_model(optimizer="adam", hidden_size1=4, hidden_size2=4):
    model = Sequential()
    model.add(Dense(hidden_size1, kernel_initializer='normal', activation='relu', input_dim=4))
    model.add(Dense(hidden_size2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(3, kernel_initializer='normal', activation='softmax'))
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    return model

clf = KerasClassifier(make_model)

param_grid = {'epochs': [100],  
              'hidden_size1': np.arange(2,12,2),  # epochs is fit parameter, not in make_model!
              'hidden_size2': np.arange(2,12,2)}

grid = GridSearchCV(clf, param_grid=param_grid, cv=5)

In [ ]:
grid.fit(X_train, y_train)

Epoch 1/100
89/89 [==============================] - 2s - loss: 1.0987 - acc: 0.2360     
Epoch 2/100
89/89 [==============================] - 0s - loss: 1.0986 - acc: 0.3596     
Epoch 3/100
89/89 [==============================] - 0s - loss: 1.0986 - acc: 0.3596     
Epoch 4/100
89/89 [==============================] - 0s - loss: 1.0985 - acc: 0.3596     
Epoch 5/100
89/89 [==============================] - 0s - loss: 1.0985 - acc: 0.3596     
Epoch 6/100
89/89 [==============================] - 0s - loss: 1.0984 - acc: 0.3596     
Epoch 7/100
89/89 [==============================] - 0s - loss: 1.0984 - acc: 0.3596     
Epoch 8/100
89/89 [==============================] - 0s - loss: 1.0983 - acc: 0.3596     
Epoch 9/100
89/89 [==============================] - 0s - loss: 1.0983 - acc: 0.3596     
Epoch 10/100
89/89 [==============================] - 0s - loss: 1.0983 - acc: 0.3596     
Epoch 11/100
89/89 [==============================] - 0s - loss: 1.0982 - acc: 0.3596     
Epoch 12

In [65]:
res = pd.DataFrame(grid.cv_results_)
res.pivot_table(index=[ "param_hidden_size1","param_hidden_size2"],
                values=['mean_train_score', "mean_test_score"])

mean_test_score  mean_train_score
param_hidden_size1 param_hidden_size2                                   
2                  2                          0.419643          0.506092
                   4                          0.473214          0.569039
                   6                          0.553571          0.633483
                   8                          0.491071          0.571261
                   10                         0.535714          0.554057
4                  2                          0.321429          0.435705
                   4                          0.687500          0.687391
                   6                          0.687500          0.687391
                   8                          0.616071          0.643171
                   10                         0.687500          0.689613
6                  2                          0.562500          0.635705
                   4                          0.491071          0.598252
                   6                          0.687500          0.685144
                   8                          0.723214          0.727416
                   10                         0.696429          0.687391
8                  2                          0.535714          0.554057
                   4                          0.589286          0.620699
                   6                          0.669643          0.689838
                   8                          0.839286          0.861523
                   10                         0.830357          0.825643
10                 2                          0.535714          0.554057
                   4                          0.642857          0.658652
                   6                          0.750000          0.740724
                   8                          0.696429          0.721099
                   10                         0.821429          0.875431

In [76]:
best_model = grid.best_estimator_
score = best_model.score(X_test, y_test)
print("Test score: {:.3f}".format(score))
#print("Test Accuracy: {:.3f}".format(score[1]))

32/38 [========================>.....] - ETA: 0sTest score: 0.711


In [74]:
score

0.71052631265238708